# Введение в MapReduce модель на Python


In [1]:
from typing import NamedTuple # requires python 3.6+
from typing import Iterator

In [2]:
def MAP(_, row:NamedTuple):
  if (row.gender == 'female'):
    yield (row.age, row)
    
def REDUCE(age:str, rows:Iterator[NamedTuple]):
  sum = 0
  count = 0
  for row in rows:
    sum += row.social_contacts
    count += 1
  if (count > 0):
    yield (age, sum/count)
  else:
    yield (age, 0)

Модель элемента данных

In [3]:
class User(NamedTuple):
  id: int
  age: str
  social_contacts: int
  gender: str

In [4]:
input_collection = [
    User(id=0, age=55, gender='male', social_contacts=20),
    User(id=1, age=25, gender='female', social_contacts=240),
    User(id=2, age=25, gender='female', social_contacts=500),
    User(id=3, age=33, gender='female', social_contacts=800)
]

Функция RECORDREADER моделирует чтение элементов с диска или по сети.

In [5]:
def RECORDREADER():
  return [(u.id, u) for u in input_collection]

In [6]:
list(RECORDREADER())

[(0, User(id=0, age=55, social_contacts=20, gender='male')),
 (1, User(id=1, age=25, social_contacts=240, gender='female')),
 (2, User(id=2, age=25, social_contacts=500, gender='female')),
 (3, User(id=3, age=33, social_contacts=800, gender='female'))]

In [7]:
def flatten(nested_iterable):
  for iterable in nested_iterable:
    for element in iterable:
      yield element

In [8]:
map_output = flatten(map(lambda x: MAP(*x), RECORDREADER()))
map_output = list(map_output) # materialize
map_output

[(25, User(id=1, age=25, social_contacts=240, gender='female')),
 (25, User(id=2, age=25, social_contacts=500, gender='female')),
 (33, User(id=3, age=33, social_contacts=800, gender='female'))]

In [9]:
def groupbykey(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

In [10]:
shuffle_output = groupbykey(map_output)
shuffle_output = list(shuffle_output)
shuffle_output

[(25,
  [User(id=1, age=25, social_contacts=240, gender='female'),
   User(id=2, age=25, social_contacts=500, gender='female')]),
 (33, [User(id=3, age=33, social_contacts=800, gender='female')])]

In [11]:
reduce_output = flatten(map(lambda x: REDUCE(*x), shuffle_output))
reduce_output = list(reduce_output)
reduce_output

[(25, 370.0), (33, 800.0)]

Все действия одним конвейером!

In [12]:
list(flatten(map(lambda x: REDUCE(*x), groupbykey(flatten(map(lambda x: MAP(*x), RECORDREADER()))))))

[(25, 370.0), (33, 800.0)]

# **MapReduce**
Выделим общую для всех пользователей часть системы в отдельную функцию высшего порядка. Это наиболее простая модель MapReduce, без учёта распределённого хранения данных. 

Пользователь для решения своей задачи реализует RECORDREADER, MAP, REDUCE.

In [13]:
def flatten(nested_iterable):
  for iterable in nested_iterable:
    for element in iterable:
      yield element

def groupbykey(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

def MapReduce(RECORDREADER, MAP, REDUCE):
  return flatten(map(lambda x: REDUCE(*x), groupbykey(flatten(map(lambda x: MAP(*x), RECORDREADER())))))

## Спецификация MapReduce



```
f (k1, v1) -> (k2,v2)*
g (k2, v2*) -> (k3,v3)*
 
mapreduce ((k1,v1)*) -> (k3,v3)*
groupby ((k2,v2)*) -> (k2,v2*)*
flatten (e2**) -> e2*
 
mapreduce .map(f).flatten.groupby(k2).map(g).flatten
```




# Примеры

## SQL 

In [14]:
from typing import NamedTuple # requires python 3.6+
from typing import Iterator

class User(NamedTuple):
  id: int
  age: str
  social_contacts: int
  gender: str
    
input_collection = [
    User(id=0, age=55, gender='male', social_contacts=20),
    User(id=1, age=25, gender='female', social_contacts=240),
    User(id=2, age=25, gender='female', social_contacts=500),
    User(id=3, age=33, gender='female', social_contacts=800)
]

def MAP(_, row:NamedTuple):
  if (row.gender == 'female'):
    yield (row.age, row)
    
def REDUCE(age:str, rows:Iterator[NamedTuple]):
  sum = 0
  count = 0
  for row in rows:
    sum += row.social_contacts
    count += 1
  if (count > 0):
    yield (age, sum/count)
  else:
    yield (age, 0)
 
def RECORDREADER():
  return [(u.id, u) for u in input_collection]

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[(25, 370.0), (33, 800.0)]

## Matrix-Vector multiplication 

In [15]:
from typing import Iterator
import numpy as np

mat = np.ones((5,4))
vec = np.random.rand(4) # in-memory vector in all map tasks

def MAP(coordinates:(int, int), value:int):
  i, j = coordinates
  yield (i, value*vec[j])
 
def REDUCE(i:int, products:Iterator[NamedTuple]):
  sum = 0
  for p in products:
    sum += p
  yield (i, sum)

def RECORDREADER():
  for i in range(mat.shape[0]):
    for j in range(mat.shape[1]):
      yield ((i, j), mat[i,j])
      
output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[(0, 2.7644114811800313),
 (1, 2.7644114811800313),
 (2, 2.7644114811800313),
 (3, 2.7644114811800313),
 (4, 2.7644114811800313)]

## Inverted index 

In [16]:
from typing import Iterator

d1 = "it is what it is"
d2 = "what is it"
d3 = "it is a banana"
documents = [d1, d2, d3]

def RECORDREADER():
  for (docid, document) in enumerate(documents):
    yield ("{}".format(docid), document)
      
def MAP(docId:str, body:str):
  for word in set(body.split(' ')):
    yield (word, docId)
 
def REDUCE(word:str, docIds:Iterator[str]):
  yield (word, sorted(docIds))

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[('what', ['0', '1']),
 ('it', ['0', '1', '2']),
 ('is', ['0', '1', '2']),
 ('banana', ['2']),
 ('a', ['2'])]

## WordCount

In [17]:
from typing import Iterator

d1 = """
it is what it is
it is what it is
it is what it is"""
d2 = """
what is it
what is it"""
d3 = """
it is a banana"""
documents = [d1, d2, d3]

def RECORDREADER():
  for (docid, document) in enumerate(documents):
    for (lineid, line) in enumerate(document.split('\n')):
      yield ("{}:{}".format(docid,lineid), line)

def MAP(docId:str, line:str):
  for word in line.split(" "):  
    yield (word, 1)
 
def REDUCE(word:str, counts:Iterator[int]):
  sum = 0
  for c in counts:
    sum += c
  yield (word, sum)

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[('', 3), ('it', 9), ('is', 9), ('what', 5), ('a', 1), ('banana', 1)]

# MapReduce Distributed

Добавляется в модель фабрика RECORDREARER-ов --- INPUTFORMAT, функция распределения промежуточных результатов по партициям PARTITIONER, и функция COMBINER для частичной аггрегации промежуточных результатов до распределения по новым партициям.

In [18]:
def flatten(nested_iterable):
  for iterable in nested_iterable:
    for element in iterable:
      yield element

def groupbykey(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()
      
def groupbykey_distributed(map_partitions, PARTITIONER):
  global reducers
  partitions = [dict() for _ in range(reducers)]
  for map_partition in map_partitions:
    for (k2, v2) in map_partition:
      p = partitions[PARTITIONER(k2)]
      p[k2] = p.get(k2, []) + [v2]
  return [(partition_id, sorted(partition.items(), key=lambda x: x[0])) for (partition_id, partition) in enumerate(partitions)]
 
def PARTITIONER(obj):
  global reducers
  return hash(obj) % reducers
  
def MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, PARTITIONER=PARTITIONER, COMBINER=None):
  map_partitions = map(lambda record_reader: flatten(map(lambda k1v1: MAP(*k1v1), record_reader)), INPUTFORMAT())
  if COMBINER != None:
    map_partitions = map(lambda map_partition: flatten(map(lambda k2v2: COMBINER(*k2v2), groupbykey(map_partition))), map_partitions)
  reduce_partitions = groupbykey_distributed(map_partitions, PARTITIONER) # shuffle
  reduce_outputs = map(lambda reduce_partition: (reduce_partition[0], flatten(map(lambda reduce_input_group: REDUCE(*reduce_input_group), reduce_partition[1]))), reduce_partitions)
  
  print("{} key-value pairs were sent over a network.".format(sum([len(vs) for (k,vs) in flatten([partition for (partition_id, partition) in reduce_partitions])])))
  return reduce_outputs

## Спецификация MapReduce Distributed


```
f (k1, v1) -> (k2,v2)*
g (k2, v2*) -> (k3,v3)*
 
e1 (k1, v1)
e2 (k2, v2)
partition1 (k2, v2)*
partition2 (k2, v2*)*
 
flatmap (e1->e2*, e1*) -> partition1*
groupby (partition1*) -> partition2*

mapreduce ((k1,v1)*) -> (k3,v3)*
mapreduce .flatmap(f).groupby(k2).flatmap(g)
```



## WordCount 

In [19]:
from typing import Iterator
import numpy as np

d1 = """
it is what it is
it is what it is
it is what it is"""
d2 = """
what is it
what is it"""
d3 = """
it is a banana"""
documents = [d1, d2, d3, d1, d2, d3]

maps = 3
reducers = 2

def INPUTFORMAT():
  global maps
  
  def RECORDREADER(split):
    for (docid, document) in enumerate(split):
      for (lineid, line) in enumerate(document.split('\n')):
        yield ("{}:{}".format(docid,lineid), line)
      
  split_size =  int(np.ceil(len(documents)/maps))
  for i in range(0, len(documents), split_size):
    yield RECORDREADER(documents[i:i+split_size])

def MAP(docId:str, line:str):
  for word in line.split(" "):  
    yield (word, 1)
 
def REDUCE(word:str, counts:Iterator[int]):
  sum = 0
  for c in counts:
    sum += c
  yield (word, sum)
  
# try to set COMBINER=REDUCER and look at the number of values sent over the network 
partitioned_output = MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, COMBINER=None) 
partitioned_output = [(partition_id, list(partition)) for (partition_id, partition) in partitioned_output]
partitioned_output

56 key-value pairs were sent over a network.


[(0, [('', 6), ('a', 2), ('banana', 2), ('it', 18), ('what', 10)]),
 (1, [('is', 18)])]

## TeraSort

In [20]:
import numpy as np

input_values = np.random.rand(30)
maps = 3
reducers = 2
min_value = 0.0
max_value = 1.0

def INPUTFORMAT():
  global maps
  
  def RECORDREADER(split):
    for value in split:
        yield (value, None)
      
  split_size =  int(np.ceil(len(input_values)/maps))
  for i in range(0, len(input_values), split_size):
    yield RECORDREADER(input_values[i:i+split_size])
    
def MAP(value:int, _):
  yield (value, None)
  
def PARTITIONER(key):
  global reducers
  global max_value
  global min_value
  bucket_size = (max_value-min_value)/reducers
  bucket_id = 0
  while((key>(bucket_id+1)*bucket_size) and ((bucket_id+1)*bucket_size<max_value)):
    bucket_id += 1
  return bucket_id

def REDUCE(value:int, _):
  yield (None,value)
  
partitioned_output = MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, COMBINER=None, PARTITIONER=PARTITIONER)
partitioned_output = [(partition_id, list(partition)) for (partition_id, partition) in partitioned_output]
partitioned_output

30 key-value pairs were sent over a network.


[(0,
  [(None, 0.007035932437115644),
   (None, 0.05044462822255513),
   (None, 0.0761069163900927),
   (None, 0.12503003933406664),
   (None, 0.15754390857749156),
   (None, 0.20460535998217766),
   (None, 0.26449113085939135),
   (None, 0.2822903263193798),
   (None, 0.29598256130751455),
   (None, 0.31222413925470105),
   (None, 0.32904970840254655),
   (None, 0.33057893856771725),
   (None, 0.42869047212415234),
   (None, 0.43069210371069877)]),
 (1,
  [(None, 0.544597192752006),
   (None, 0.5826193297616951),
   (None, 0.5984408049163263),
   (None, 0.6033263353879544),
   (None, 0.6095278140938067),
   (None, 0.627460195498983),
   (None, 0.6915586458480801),
   (None, 0.7278064874205156),
   (None, 0.730193662828503),
   (None, 0.8034067567518253),
   (None, 0.8711925576896182),
   (None, 0.8839887002256163),
   (None, 0.8911652107891863),
   (None, 0.9380676723995685),
   (None, 0.9499817619723798),
   (None, 0.9969785160891388)])]

# Упражнения
Упражнения взяты из Rajaraman A., Ullman J. D. Mining of massive datasets. – Cambridge University Press, 2011.


Для выполнения заданий переопределите функции RECORDREADER, MAP, REDUCE. Для модели распределённой системы может потребоваться переопределение функций PARTITION и COMBINER.

### Максимальное значение ряда

Разработайте MapReduce алгоритм, который находит максимальное число входного списка чисел.

In [21]:
# Импорт модуля random для генерации случайных чисел
import random

# Функция MAP, которая принимает список (chunk) и возвращает максимальное значение из него
def MAP(chunk):
    return max(chunk)

# Функция REDUCE, которая принимает список результатов (results) и возвращает максимальное значение из них
def REDUCE(results):
    return max(results)

# Функция RECORDREADER, которая создает список случайных чисел заданного размера (count)
# Каждое число в диапазоне от 0 до 100
def RECORDREADER(count):
    return [random.randint(0, 100) for i in range(count)]

# Генерация записей с помощью функции RECORDREADER и их вывод
record = RECORDREADER(100)
print(record,'\n')

# Разделение записей на части (партиции) с количеством частей, равным part_count
# Каждая часть содержит part_count записей
record_partitional = [record[d:d + part_count] for d in range(0, len(record), part_count)]
print(record_partitional,'\n')

# Применение функции MAP к каждой части записей, получение списка результатов и применение функции REDUCE к этому списку
# Вывод результата - максимального значения из всех максимальных значений частей записей
print(REDUCE(list(map(lambda x: MAP(x), record_partitional))))

[81, 82, 86, 40, 18, 5, 10, 58, 28, 78, 76, 37, 29, 53, 0, 48, 51, 32, 41, 35, 11, 87, 17, 48, 86, 89, 58, 14, 27, 44, 91, 13, 16, 99, 2, 98, 5, 28, 62, 100, 99, 33, 72, 61, 91, 33, 34, 96, 48, 21, 99, 69, 54, 87, 77, 88, 91, 0, 1, 75, 31, 9, 67, 7, 39, 63, 85, 8, 63, 17, 41, 98, 75, 96, 70, 27, 81, 52, 81, 16, 55, 6, 91, 16, 92, 24, 85, 62, 50, 14, 96, 54, 63, 23, 37, 2, 2, 59, 15, 47] 

[[81, 82, 86, 40, 18], [5, 10, 58, 28, 78], [76, 37, 29, 53, 0], [48, 51, 32, 41, 35], [11, 87, 17, 48, 86], [89, 58, 14, 27, 44], [91, 13, 16, 99, 2], [98, 5, 28, 62, 100], [99, 33, 72, 61, 91], [33, 34, 96, 48, 21], [99, 69, 54, 87, 77], [88, 91, 0, 1, 75], [31, 9, 67, 7, 39], [63, 85, 8, 63, 17], [41, 98, 75, 96, 70], [27, 81, 52, 81, 16], [55, 6, 91, 16, 92], [24, 85, 62, 50, 14], [96, 54, 63, 23, 37], [2, 2, 59, 15, 47]] 

100


### Арифметическое среднее

Разработайте MapReduce алгоритм, который находит арифметическое среднее.

$$\overline{X} = \frac{1}{n}\sum_{i=0}^{n} x_i$$


In [22]:
# Импорт класса Pool из модуля multiprocessing для параллельного выполнения задач
from multiprocessing import Pool

# Функция MAP, которая принимает часть данных (chunk) и возвращает кортеж с ключом 1 и значением chunk
def MAP(chunk):
    return (1, chunk)

# Функция REDUCE, которая принимает ключ и итератор с числами, а затем вычисляет среднее значение чисел
# Результат вычислений возвращается в виде генератора
def REDUCE(key, nums):
    sum = 0
    count = 0
    for n in nums:
        sum += n
        count += 1
    yield sum / count

# Функция RECORDREADER, которая создает список случайных чисел заданного размера (count)
# Каждое число находится в диапазоне от 0 до 100
def RECORDREADER(count):
    return [random.randint(0, 100) for i in range(count)]

# Применение функции MAP к списку записей, созданному с помощью функции RECORDREADER
map_output = list(map(lambda x: MAP(x), RECORDREADER(100)))
print(map_output)

# Группировка результатов функции MAP по ключу
# Результат сохраняется в виде списка кортежей, каждый кортеж содержит ключ и список значений
shuffle_output = groupbykey(map_output)
shuffle_output = list(shuffle_output)
print(shuffle_output)

# Применение функции REDUCE к результатам группировки
# Результатом является список, содержащий средние значения для каждой группы значений
reduce_output = list(flatten(map(lambda x: REDUCE(*x), shuffle_output)))
print(reduce_output)

[(1, 88), (1, 71), (1, 88), (1, 69), (1, 44), (1, 42), (1, 85), (1, 2), (1, 36), (1, 25), (1, 80), (1, 22), (1, 74), (1, 8), (1, 79), (1, 82), (1, 30), (1, 58), (1, 23), (1, 74), (1, 76), (1, 74), (1, 79), (1, 62), (1, 71), (1, 53), (1, 73), (1, 38), (1, 48), (1, 3), (1, 42), (1, 41), (1, 100), (1, 23), (1, 54), (1, 39), (1, 85), (1, 37), (1, 65), (1, 7), (1, 6), (1, 61), (1, 7), (1, 37), (1, 100), (1, 8), (1, 14), (1, 1), (1, 7), (1, 69), (1, 23), (1, 58), (1, 2), (1, 11), (1, 16), (1, 85), (1, 12), (1, 1), (1, 82), (1, 63), (1, 38), (1, 66), (1, 62), (1, 95), (1, 39), (1, 52), (1, 78), (1, 60), (1, 14), (1, 26), (1, 70), (1, 59), (1, 11), (1, 90), (1, 91), (1, 17), (1, 13), (1, 15), (1, 73), (1, 94), (1, 76), (1, 56), (1, 100), (1, 38), (1, 66), (1, 67), (1, 35), (1, 8), (1, 99), (1, 60), (1, 3), (1, 7), (1, 87), (1, 59), (1, 14), (1, 22), (1, 100), (1, 99), (1, 33), (1, 84)]
[(1, [88, 71, 88, 69, 44, 42, 85, 2, 36, 25, 80, 22, 74, 8, 79, 82, 30, 58, 23, 74, 76, 74, 79, 62, 71, 53, 7

### GroupByKey на основе сортировки

Реализуйте groupByKey на основе сортировки, проверьте его работу на примерах

In [23]:
# Функция group_by_key принимает итерируемый объект и группирует его элементы по ключу
def group_by_key(iterable):
    # Сортируем данные по ключу
    sorted_data = sorted(iterable, key=lambda x: x[0])
    # Создаем словарь для хранения сгруппированных данных
    grouped_data = {}
    # Проходимся по отсортированным данным и добавляем их в словарь
    for key, value in sorted_data:
        if key not in grouped_data:
            grouped_data[key] = [value]
        else:
            grouped_data[key].append(value)
    # Преобразуем словарь в список кортежей и возвращаем результат
    result = []
    for key, value in grouped_data.items():
        result.append((key, value))
    return result

# Функция MAP принимает число и возвращает кортеж с фиксированным ключом (1) и переданным числом
def MAP(num):
    return (1, num)

# Функция REDUCE вычисляет среднее значение чисел в итераторе
def REDUCE(key, nums: Iterator[NamedTuple]):
    sum = 0
    count = 0
    for n in nums:
        sum += n
        count += 1
    # Генерируем среднее значение и возвращаем его как генератор
    yield sum / count

# Функция RECORDREADER генерирует список случайных чисел заданного размера
def RECORDREADER(count):
    return [random.randint(0, 100) for i in range(count)]

# Генерация записей с помощью функции RECORDREADER и их обработка с использованием функций MAP, group_by_key и REDUCE
map_output = list(map(lambda x: MAP(x), RECORDREADER(100)))
print(map_output)

shuffle_output = group_by_key(map_output)
shuffle_output = list(shuffle_output)
print(shuffle_output)

reduce_output = list(flatten(map(lambda x: REDUCE(*x), shuffle_output)))
print(reduce_output)
     

[(1, 1), (1, 83), (1, 13), (1, 58), (1, 30), (1, 31), (1, 3), (1, 32), (1, 51), (1, 72), (1, 18), (1, 83), (1, 11), (1, 60), (1, 13), (1, 70), (1, 57), (1, 8), (1, 89), (1, 22), (1, 16), (1, 87), (1, 46), (1, 38), (1, 35), (1, 63), (1, 30), (1, 43), (1, 80), (1, 27), (1, 92), (1, 32), (1, 13), (1, 8), (1, 60), (1, 18), (1, 39), (1, 67), (1, 36), (1, 52), (1, 92), (1, 66), (1, 31), (1, 13), (1, 69), (1, 58), (1, 55), (1, 52), (1, 90), (1, 11), (1, 36), (1, 16), (1, 81), (1, 64), (1, 54), (1, 68), (1, 72), (1, 87), (1, 13), (1, 9), (1, 93), (1, 82), (1, 83), (1, 16), (1, 64), (1, 0), (1, 39), (1, 64), (1, 73), (1, 56), (1, 61), (1, 54), (1, 50), (1, 46), (1, 42), (1, 28), (1, 35), (1, 32), (1, 63), (1, 77), (1, 40), (1, 75), (1, 42), (1, 37), (1, 3), (1, 43), (1, 94), (1, 62), (1, 97), (1, 83), (1, 43), (1, 58), (1, 11), (1, 93), (1, 60), (1, 83), (1, 62), (1, 78), (1, 30), (1, 4)]
[(1, [1, 83, 13, 58, 30, 31, 3, 32, 51, 72, 18, 83, 11, 60, 13, 70, 57, 8, 89, 22, 16, 87, 46, 38, 35, 63, 

### Drop duplicates (set construction, unique elements, distinct)

Реализуйте распределённую операцию исключения дубликатов

In [24]:
# Создание списка пользователей с различными атрибутами, такими как возраст, пол и количество социальных контактов
input_collection = [
    User(id=0, age=54, gender="male", social_contacts=20),
    User(id=0, age=25, gender="female", social_contacts=240),
    User(id=1, age=27, gender="male", social_contacts=642),
    User(id=2, age=16, gender="male", social_contacts=123),
    User(id=2, age=35, gender="female", social_contacts=247),
    User(id=3, age=31, gender="male", social_contacts=521),
    User(id=3, age=32, gender="female", social_contacts=753),
]

# Функция, которая разглаживает вложенные итерируемые объекты
def flatten(nested_iterable):
    for iterable in nested_iterable:
        for element in iterable:
            yield element

# Функция, которая группирует элементы по ключу
def groupbykey(iterable):
    t = {}
    for k2, v2 in iterable:
        t[k2] = t.get(k2, []) + [v2]
    return t.items()

# Функция, которая выполняет группировку данных с использованием нескольких редукторов
def groupbykey_distributed(map_partitions, PARTITIONER):
    global reducers
    partitions = [dict() for _ in range(reducers)]
    for map_partition in map_partitions:
        for k2, v2 in map_partition:
            p = partitions[PARTITIONER(k2)]
            p[k2] = p.get(k2, []) + [v2]
    return [
        (partition_id, sorted(partition.items(), key=lambda x: x[0]))
        for (partition_id, partition) in enumerate(partitions)
    ]

# Функция, которая определяет, какой редуктор будет обрабатывать данные, основываясь на хэше ключа
def PARTITIONER(obj):
    global reducers
    return hash(obj) % reducers

# Функция, которая запускает процесс MapReduce, принимая на вход INPUTFORMAT, MAP, REDUCE, PARTITIONER и COMBINER
def MapReduceDistributed(
    INPUTFORMAT, MAP, REDUCE, PARTITIONER=PARTITIONER, COMBINER=None
):
    map_partitions = map(
        lambda record_reader: flatten(map(lambda k1v1: MAP(*k1v1), record_reader)),
        INPUTFORMAT(),
    )
    if COMBINER != None:
        map_partitions = map(
            lambda map_partition: flatten(
                map(lambda k2v2: COMBINER(*k2v2), groupbykey(map_partition))
            ),
            map_partitions,
        )
    reduce_partitions = groupbykey_distributed(map_partitions, PARTITIONER)  # shuffle
    reduce_outputs = map(
        lambda reduce_partition: (
            reduce_partition[0],
            flatten(
                map(
                    lambda reduce_input_group: REDUCE(*reduce_input_group),
                    reduce_partition[1],
                )
            ),
        ),
        reduce_partitions,
    )

    print(
        "{} key-value pairs were sent over a network.".format(
            sum(
                [
                    len(vs)
                    for (k, vs) in flatten(
                        [partition for (partition_id, partition) in reduce_partitions]
                    )
                ]
            )
        )
    )
    return reduce_outputs

maps = 3
reducers = 2

# Функция, которая разделяет входные данные между вычислительными узлами
def INPUTFORMAT():
    global maps

    def RECORDREADER(split):
        for userindex, user in enumerate(split):
            yield (userindex, user)

    # Разделение массива пользователей между вычислительными узлами
    split_size = int(np.ceil(len(input_collection) / maps))
    for i in range(0, len(input_collection), split_size):
        yield RECORDREADER(input_collection[i : i + split_size])


# Функция, которая принимает запись пользователя и создает пары ключ-значение, где ключом является id пользователя
def MAP_UNIQUE(_, user: str):
    yield (user.id, user)


# Функция, которая принимает группу пользователей с одинаковым id и возвращает только первого пользователя из группы
def REDUCE_UNIQUE(user_id: int, users: Iterator[int]):
    yield users[0]


# try to set COMBINER=REDUCER and look at the number of values sent over the network
partitioned_output = MapReduceDistributed(
    INPUTFORMAT, MAP_UNIQUE, REDUCE_UNIQUE, COMBINER=None
)
partitioned_output = [
    (partition_id, list(partition)) for (partition_id, partition) in partitioned_output
]
partitioned_output

7 key-value pairs were sent over a network.


[(0,
  [User(id=0, age=54, social_contacts=20, gender='male'),
   User(id=2, age=16, social_contacts=123, gender='male')]),
 (1,
  [User(id=1, age=27, social_contacts=642, gender='male'),
   User(id=3, age=31, social_contacts=521, gender='male')])]

#Операторы реляционной алгебры
### Selection (Выборка)

**The Map Function**: Для  каждого кортежа $t \in R$ вычисляется истинность предиката $C$. В случае истины создаётся пара ключ-значение $(t, t)$. В паре ключ и значение одинаковы, равны $t$.

**The Reduce Function:** Роль функции Reduce выполняет функция идентичности, которая возвращает то же значение, что получила на вход.



In [25]:
from collections import defaultdict  # Импортируем необходимый модуль defaultdict из collections

def MAP(el_list):
    mapped_result = defaultdict(list)  # Создаем defaultdict для хранения отображенных значений
    for t in el_list:
        if C(t):  # Если условие C(t) истинно для элемента t
            mapped_result[t].append(t)  # Добавляем элемент t в список, связанный с ключом t
    return mapped_result.items()  # Возвращаем отображенный результат в виде списка кортежей

def REDUCE(mapped_items):
    reduced_result = []
    print(mapped_items)  # Выводим отображенные элементы для отладки
    for values in mapped_items:
        for value in values:  # Перебираем каждый элемент в значениях
            reduced_result.append(value[0])  # Добавляем первый элемент кортежа в редуцированный результат
    return reduced_result

def C(t):
    return t[0] % 2 == 0  # Возвращаем True, если первый элемент кортежа t четный

# Генерируем список кортежей, каждый из которых содержит три случайных числа от 0 до 100
def RECORDREADER(count):
    return [(random.randint(0, 100), random.randint(0, 100), random.randint(0, 100)) for i in range(count)]

record = RECORDREADER(100)  # Создаем список из 100 случайных записей
print(record)  # Выводим список записей

# Разбиваем список записей на подсписки длиной part_count
part_count = 5
record_partitional = [record[d:d + part_count] for d in range(0, len(record), part_count)]

print(record_partitional)  # Выводим разбиение записей на подсписки

# Применяем MAP к каждому подсписку и затем редуцируем результаты
print(REDUCE(list(map(lambda x: MAP(x), record_partitional))))

[(36, 1, 8), (29, 46, 81), (91, 6, 41), (83, 39, 84), (76, 14, 93), (43, 6, 12), (17, 96, 89), (41, 87, 57), (15, 91, 100), (45, 38, 90), (96, 39, 36), (31, 27, 66), (38, 22, 82), (35, 35, 62), (60, 21, 57), (3, 11, 52), (20, 95, 22), (78, 99, 49), (71, 0, 44), (55, 32, 4), (97, 10, 100), (51, 20, 42), (16, 23, 86), (3, 18, 50), (19, 74, 95), (11, 52, 11), (100, 34, 61), (13, 43, 28), (32, 67, 0), (46, 100, 92), (88, 5, 25), (6, 78, 93), (97, 65, 64), (24, 60, 57), (59, 79, 54), (73, 66, 54), (38, 70, 98), (95, 75, 21), (42, 67, 28), (38, 4, 48), (61, 35, 36), (48, 36, 68), (96, 77, 96), (5, 28, 46), (4, 72, 65), (80, 13, 66), (59, 32, 28), (87, 90, 78), (89, 53, 14), (92, 15, 74), (92, 27, 77), (21, 80, 93), (4, 16, 12), (89, 35, 6), (21, 54, 58), (45, 24, 24), (23, 32, 59), (25, 84, 88), (13, 81, 70), (12, 76, 70), (15, 85, 10), (1, 62, 64), (6, 79, 75), (33, 39, 25), (7, 2, 12), (26, 69, 22), (53, 10, 33), (64, 48, 44), (34, 16, 47), (94, 15, 71), (44, 45, 14), (59, 51, 79), (78, 0,

### Projection (Проекция)

Проекция на множество атрибутов $S$.

**The Map Function:** Для каждого кортежа $t \in R$ создайте кортеж $t′$, исключая  из $t$ те значения, атрибуты которых не принадлежат  $S$. Верните пару $(t′, t′)$.

**The Reduce Function:** Для каждого ключа $t′$, созданного любой Map задачей, вы получаете одну или несколько пар $(t′, t′)$. Reduce функция преобразует $(t′, [t′, t′, . . . , t′])$ в $(t′, t′)$, так, что для ключа $t′$ возвращается одна пара  $(t′, t′)$.

In [26]:
S = set([1, 12, 56])  # Определяем множество S

def MAP(t):
    res_list = []
    for el in t:  # Перебираем элементы входного кортежа t
        if el in S:  # Если элемент есть в множестве S
            res_list.append(el)  # Добавляем его в список res_list
    res = tuple(res_list)  # Преобразуем список в кортеж
    return (res, res)  # Возвращаем кортеж из двух элементов, где оба одинаковы

def REDUCE(key, values):
    return (key, key)  # Возвращаем ключ

# Генерируем список кортежей, каждый из которых содержит три случайных числа от 0 до 100
def RECORDREADER(count):
    return [(random.randint(0, 100), random.randint(0, 100), random.randint(0, 100)) for i in range(count)]

def group_by_key(iterable):
    t = {}
    for (k2, v2) in iterable:  # Перебираем ключи и значения во входном итерируемом объекте
        t[k2] = t.get(k2, []) + [v2]  # Добавляем значение v2 в список, связанный с ключом k2 в словаре t
    return t.items()  # Возвращаем элементы словаря как список кортежей

record = RECORDREADER(100)  # Создаем список из 100 случайных записей
print(record)  # Выводим список записей

map_output = list(map(lambda x: MAP(x), RECORDREADER(100)))  # Применяем MAP к каждой записи и сохраняем результат
print(map_output)  # Выводим результаты отображения

shuffle_output = group_by_key(map_output)  # Группируем результаты отображения по ключу
shuffle_output = list(shuffle_output)  # Преобразуем результат группировки в список
print(shuffle_output)  # Выводим результат группировки

reduce_output = list(map(lambda x: REDUCE(*x)[0], shuffle_output))  # Применяем REDUCE к результату группировки
print(reduce_output)  # Выводим редуцированный результат
     

[(79, 59, 25), (44, 6, 37), (8, 18, 96), (53, 0, 95), (81, 55, 84), (41, 56, 26), (43, 68, 75), (82, 74, 23), (35, 93, 55), (89, 38, 25), (85, 82, 72), (63, 47, 7), (59, 96, 75), (100, 70, 16), (85, 67, 6), (96, 36, 98), (30, 54, 87), (50, 48, 81), (48, 58, 66), (64, 61, 28), (0, 93, 53), (55, 33, 0), (40, 94, 2), (33, 43, 22), (90, 57, 30), (28, 29, 81), (90, 100, 57), (59, 41, 59), (74, 68, 32), (13, 72, 17), (75, 89, 32), (17, 97, 25), (29, 92, 52), (28, 84, 4), (49, 24, 83), (41, 84, 81), (66, 90, 21), (56, 4, 21), (18, 73, 28), (38, 63, 1), (27, 50, 81), (11, 36, 24), (70, 96, 49), (20, 70, 29), (59, 91, 22), (93, 50, 69), (48, 61, 68), (37, 43, 42), (8, 97, 94), (4, 69, 72), (89, 52, 62), (3, 6, 24), (49, 37, 73), (76, 46, 77), (80, 54, 67), (65, 72, 36), (73, 34, 92), (28, 36, 66), (45, 48, 31), (24, 65, 52), (62, 21, 69), (7, 74, 51), (61, 7, 80), (48, 75, 16), (70, 68, 25), (79, 90, 93), (60, 94, 18), (21, 30, 87), (8, 19, 28), (92, 77, 32), (33, 2, 16), (4, 17, 96), (87, 21, 

### Union (Объединение)

**The Map Function:** Превратите каждый входной кортеж $t$ в пару ключ-значение $(t, t)$.

**The Reduce Function:** С каждым ключом $t$ будет ассоциировано одно или два значений. В обоих случаях создайте $(t, t)$ в качестве выходного значения.

In [27]:
def MAP(t):
    return (t, t)  # Возвращаем входное значение в форме кортежа (t, t)

def REDUCE(key, values: Iterator[NamedTuple]):
    return (key, key)  # Возвращаем ключ

# Генерируем список кортежей, каждый из которых содержит три случайных числа от 0 до 100
def RECORDREADER(count):
    return [(random.randint(0, 100), random.randint(0, 100), random.randint(0, 100)) for i in range(count)]

def group_by_key(iterable):
    t = {}  # Создаем пустой словарь
    for (k2, v2) in iterable:  # Перебираем ключи и значения во входном итерируемом объекте
        t[k2] = t.get(k2, []) + [v2]  # Добавляем значение v2 в список, связанный с ключом k2 в словаре t
    return t.items()  # Возвращаем элементы словаря как список кортежей

record = RECORDREADER(100)  # Создаем список из 100 случайных записей
print(record)  # Выводим список записей

map_output = list(map(lambda x: MAP(x), RECORDREADER(100)))  # Применяем MAP к каждой записи и сохраняем результат
print(map_output)  # Выводим результаты отображения

shuffle_output = group_by_key(map_output)  # Группируем результаты отображения по ключу
shuffle_output = list(shuffle_output)  # Преобразуем результат группировки в список
print(shuffle_output)  # Выводим результат группировки

reduce_output = list(map(lambda x: REDUCE(*x)[0], shuffle_output))  # Применяем REDUCE к результату группировки
print(reduce_output)  # Выводим редуцированный результат

[(70, 65, 91), (17, 39, 10), (92, 26, 39), (54, 1, 37), (40, 24, 13), (22, 23, 65), (58, 86, 71), (75, 5, 41), (52, 13, 21), (60, 99, 33), (74, 33, 57), (26, 6, 40), (48, 5, 97), (62, 35, 83), (99, 98, 50), (42, 25, 22), (66, 0, 13), (48, 44, 8), (71, 98, 75), (59, 42, 63), (29, 58, 82), (77, 24, 3), (1, 98, 8), (8, 38, 2), (29, 32, 13), (31, 22, 98), (43, 14, 41), (54, 85, 63), (92, 26, 40), (86, 11, 27), (35, 54, 80), (69, 4, 11), (3, 4, 89), (6, 22, 75), (81, 47, 1), (67, 22, 61), (14, 94, 47), (36, 12, 63), (20, 42, 93), (6, 100, 12), (39, 84, 7), (44, 74, 68), (86, 74, 2), (23, 27, 6), (32, 1, 39), (79, 21, 53), (75, 62, 6), (36, 55, 46), (12, 7, 77), (45, 31, 54), (12, 58, 72), (36, 48, 7), (42, 3, 53), (25, 3, 40), (42, 90, 89), (28, 44, 50), (60, 88, 9), (4, 10, 58), (56, 37, 47), (2, 65, 80), (40, 50, 0), (32, 82, 54), (44, 67, 72), (35, 61, 6), (83, 89, 86), (15, 44, 9), (86, 14, 55), (44, 69, 64), (64, 74, 61), (33, 50, 42), (92, 30, 94), (47, 9, 75), (45, 44, 8), (96, 55, 2

### Intersection (Пересечение)

**The Map Function:** Превратите каждый кортеж $t$ в пары ключ-значение $(t, t)$.

**The Reduce Function:** Если для ключа $t$ есть список из двух элементов $[t, t]$ $-$ создайте пару $(t, t)$. Иначе, ничего не создавайте.

In [28]:
def MAP(t):
    return (t, t)  # Возвращаем входное значение в форме кортежа (t, t)

def REDUCE(key, values: Iterator[NamedTuple]):
    if len(values) == 2:  # Если количество значений равно 2
        return (key, key)  # Возвращаем ключ
    # если условие не выполняется, функция не возвращает ничего, что приводит к возвращению значения None

# Генерируем список кортежей, каждый из которых содержит два случайных числа от 0 до 3
def RECORDREADER(count):
    return [(random.randint(0, 3), random.randint(0, 3)) for i in range(count)]

def group_by_key(iterable):
    t = {}  # Создаем пустой словарь
    for (k2, v2) in iterable:  # Перебираем ключи и значения во входном итерируемом объекте
        t[k2] = t.get(k2, []) + [v2]  # Добавляем значение v2 в список, связанный с ключом k2 в словаре t
    return t.items()  # Возвращаем элементы словаря как список кортежей

record = RECORDREADER(100)  # Создаем список из 100 случайных записей
print(record)  # Выводим список записей

map_output = list(map(lambda x: MAP(x), RECORDREADER(100)))  # Применяем MAP к каждой записи и сохраняем результат
print(map_output)  # Выводим результаты отображения

shuffle_output = group_by_key(map_output)  # Группируем результаты отображения по ключу
shuffle_output = list(shuffle_output)  # Преобразуем результат группировки в список
print(shuffle_output)  # Выводим результат группировки

# Применяем REDUCE к результату группировки и сохраняем результат
# Затем фильтруем результаты, исключая значения None, и оставляем только первый элемент каждого кортежа
reduce_output = [el[0] for el in list(map(lambda x: REDUCE(*x), shuffle_output)) if el is not None]
print(reduce_output)  # Выводим редуцированный результат

[(0, 0), (3, 0), (1, 1), (0, 3), (0, 2), (2, 2), (2, 0), (1, 1), (0, 3), (1, 1), (1, 1), (1, 2), (0, 1), (2, 1), (0, 1), (1, 1), (3, 1), (3, 1), (1, 0), (0, 2), (2, 2), (0, 2), (1, 1), (0, 0), (0, 0), (1, 3), (2, 0), (3, 2), (2, 0), (2, 0), (0, 2), (2, 3), (0, 0), (0, 2), (2, 1), (0, 3), (0, 3), (3, 3), (0, 3), (2, 2), (0, 3), (2, 0), (2, 2), (0, 2), (2, 2), (2, 3), (0, 1), (3, 0), (2, 1), (2, 0), (2, 3), (3, 2), (1, 0), (3, 3), (0, 0), (1, 0), (0, 1), (1, 3), (2, 3), (2, 0), (3, 2), (0, 1), (3, 2), (1, 1), (0, 3), (0, 2), (0, 0), (2, 3), (0, 2), (0, 0), (1, 1), (0, 1), (3, 1), (3, 3), (0, 2), (0, 0), (2, 0), (2, 1), (1, 2), (2, 3), (3, 3), (2, 0), (2, 3), (2, 2), (1, 0), (2, 2), (3, 0), (3, 2), (0, 2), (2, 2), (1, 0), (2, 0), (1, 3), (2, 3), (3, 1), (2, 2), (3, 1), (0, 3), (2, 0), (3, 0)]
[((1, 3), (1, 3)), ((1, 2), (1, 2)), ((2, 3), (2, 3)), ((2, 1), (2, 1)), ((0, 0), (0, 0)), ((3, 2), (3, 2)), ((2, 2), (2, 2)), ((3, 3), (3, 3)), ((3, 2), (3, 2)), ((1, 1), (1, 1)), ((0, 1), (0, 1)), 

### Difference (Разница)

**The Map Function:** Для кортежа $t \in R$, создайте пару $(t, R)$, и для кортежа $t \in S$, создайте пару $(t, S)$. Задумка заключается в том, чтобы значение пары было именем отношения $R$ or $S$, которому принадлежит кортеж (а лучше, единичный бит, по которому можно два отношения различить $R$ or $S$), а не весь набор атрибутов отношения.

**The Reduce Function:** Для каждого ключа $t$, если соответствующее значение является списком $[R]$, создайте пару $(t, t)$. В иных случаях не предпринимайте действий.

In [32]:
rels = [1, 2]  # Определяем список rels

class Tuple:
    def __init__(self, data: tuple, rel_id: int):
        self.data = data  # Содержит данные
        self.rel_id = rel_id  # Содержит идентификатор отношения

def get_random_tuple(count):
    data = tuple([(random.randint(0, 3), random.randint(0, 3)) for i in range(count)])  # Генерируем случайный кортеж данных
    rel_id = rels[random.randint(0, len(rels) - 1)]  # Выбираем случайный идентификатор отношения из списка rels
    return Tuple(data, rel_id)  # Возвращаем объект Tuple с сгенерированными данными и идентификатором отношения

def RECORDREADER(count):
    return [get_random_tuple(3) for i in range(count)]  # Генерируем список Tuple с помощью get_random_tuple

def MAP(t: Tuple):
    return (t.data, t.rel_id)  # Возвращает кортеж, содержащий данные и идентификатор отношения из объекта Tuple

def REDUCE(key, values: Iterator[NamedTuple]):
    if values == [rels[0]]:  # Если значения равны первому элементу списка rels
        return (key, key)  # Возвращаем ключ
    # если условие не выполняется, функция не возвращает ничего, что приводит к возвращению значения None

def group_by_key(iterable):
    t = {}  # Создаем пустой словарь
    for (k2, v2) in iterable:  # Перебираем ключи и значения во входном итерируемом объекте
        t[k2] = t.get(k2, []) + [v2]  # Добавляем значение v2 в список, связанный с ключом k2 в словаре t
    return t.items()  # Возвращаем элементы словаря как список кортежей

record = RECORDREADER(100)  # Создаем список из 100 случайных записей
print(record)  # Выводим список записей

map_output = list(map(lambda x: MAP(x), RECORDREADER(100)))  # Применяем MAP к каждой записи и сохраняем результат
print(map_output)  # Выводим результаты отображения

shuffle_output = group_by_key(map_output)  # Группируем результаты отображения по ключу
shuffle_output = list(shuffle_output)  # Преобразуем результат группировки в список
print(shuffle_output)  # Выводим результат группировки

# Применяем REDUCE к результату группировки и сохраняем результат
# Затем фильтруем результаты, исключая значения None, и оставляем только первый элемент каждого кортежа
reduce_output = [el[0] for el in list(map(lambda x: REDUCE(*x), shuffle_output)) if el is not None]
print(reduce_output)  # Выводим редуцированный результат

[<__main__.Tuple object at 0x000001EB5C1E2F10>, <__main__.Tuple object at 0x000001EB5D205C70>, <__main__.Tuple object at 0x000001EB5D2054F0>, <__main__.Tuple object at 0x000001EB5C20A790>, <__main__.Tuple object at 0x000001EB5C20A760>, <__main__.Tuple object at 0x000001EB5D245EE0>, <__main__.Tuple object at 0x000001EB5D2450D0>, <__main__.Tuple object at 0x000001EB5D245100>, <__main__.Tuple object at 0x000001EB5D2456D0>, <__main__.Tuple object at 0x000001EB5D245790>, <__main__.Tuple object at 0x000001EB5D2457F0>, <__main__.Tuple object at 0x000001EB5D245850>, <__main__.Tuple object at 0x000001EB5D2458B0>, <__main__.Tuple object at 0x000001EB5D245400>, <__main__.Tuple object at 0x000001EB5D245FA0>, <__main__.Tuple object at 0x000001EB5D245130>, <__main__.Tuple object at 0x000001EB5D245F70>, <__main__.Tuple object at 0x000001EB5D245DF0>, <__main__.Tuple object at 0x000001EB5D245E80>, <__main__.Tuple object at 0x000001EB5D245550>, <__main__.Tuple object at 0x000001EB5D245310>, <__main__.Tu

### Natural Join

**The Map Function:** Для каждого кортежа $(a, b)$ отношения $R$, создайте пару $(b,(R, a))$. Для каждого кортежа $(b, c)$ отношения $S$, создайте пару $(b,(S, c))$.

**The Reduce Function:** Каждый ключ $b$ будет асоциирован со списком пар, которые принимают форму либо $(R, a)$, либо $(S, c)$. Создайте все пары, одни, состоящие из  первого компонента $R$, а другие, из первого компонента $S$, то есть $(R, a)$ и $(S, c)$. На выходе вы получаете последовательность пар ключ-значение из списков ключей и значений. Ключ не нужен. Каждое значение, это тройка $(a, b, c)$ такая, что $(R, a)$ и $(S, c)$ это принадлежат входному списку значений.

In [33]:
rels = [1, 2]  # Определяем список rels

class Tuple:
    def __init__(self, data: tuple, rel_id: int):
        self.data = data  # Содержит данные
        self.rel_id = rel_id  # Содержит идентификатор отношения

def get_random_tuple():
    data = (random.randint(0, 3), random.randint(0, 3))  # Генерируем случайный кортеж данных
    rel_id = rels[random.randint(0, len(rels) - 1)]  # Выбираем случайный идентификатор отношения из списка rels
    return Tuple(data, rel_id)  # Возвращаем объект Tuple с сгенерированными данными и идентификатором отношения

def RECORDREADER(count):
    return [get_random_tuple() for i in range(count)]  # Генерируем список Tuple с помощью get_random_tuple

def MAP(t: Tuple):
    if t.rel_id == rels[0]:  # Если идентификатор отношения равен первому элементу списка rels
        return (t.data[1], (t.rel_id, t.data[0]))  # Возвращаем кортеж, содержащий второй элемент данных и пару (идентификатор отношения, первый элемент данных)
    else:
        return (t.data[0], (t.rel_id, t.data[1]))  # Возвращаем кортеж, содержащий первый элемент данных и пару (идентификатор отношения, второй элемент данных)

def REDUCE(key, values: Iterator[NamedTuple]):
    res = []  # Создаем пустой список для результатов
    for v in values:  # Перебираем значения
        res.append((v[0], key, v[1]))  # Добавляем кортеж в формате (значение, ключ, значение) в список res
    return res  # Возвращаем список результатов

def group_by_key(iterable):
    t = {}  # Создаем пустой словарь
    for (k2, v2) in iterable:  # Перебираем ключи и значения во входном итерируемом объекте
        t[k2] = t.get(k2, []) + [v2]  # Добавляем значение v2 в список, связанный с ключом k2 в словаре t
    return t.items()  # Возвращаем элементы словаря как список кортежей

record = RECORDREADER(100)  # Создаем список из 100 случайных записей
print(record)  # Выводим список записей

map_output = list(map(lambda x: MAP(x), RECORDREADER(100)))  # Применяем MAP к каждой записи и сохраняем результат
print(map_output)  # Выводим результаты отображения

shuffle_output = group_by_key(map_output)  # Группируем результаты отображения по ключу
shuffle_output = list(shuffle_output)  # Преобразуем результат группировки в список
print(shuffle_output)  # Выводим результат группировки

reduce_output = list(map(lambda x: REDUCE(*x), shuffle_output))  # Применяем REDUCE к результату группировки и сохраняем результат
print(reduce_output)  # Выводим редуцированный результат

[<__main__.Tuple object at 0x000001EB5D205520>, <__main__.Tuple object at 0x000001EB5D212CA0>, <__main__.Tuple object at 0x000001EB5D212250>, <__main__.Tuple object at 0x000001EB5D212A30>, <__main__.Tuple object at 0x000001EB5D212130>, <__main__.Tuple object at 0x000001EB5D212310>, <__main__.Tuple object at 0x000001EB5D212850>, <__main__.Tuple object at 0x000001EB5D212220>, <__main__.Tuple object at 0x000001EB5D212820>, <__main__.Tuple object at 0x000001EB5D2121F0>, <__main__.Tuple object at 0x000001EB5D212760>, <__main__.Tuple object at 0x000001EB5D212BE0>, <__main__.Tuple object at 0x000001EB5D212970>, <__main__.Tuple object at 0x000001EB5D2120D0>, <__main__.Tuple object at 0x000001EB5C1F9A00>, <__main__.Tuple object at 0x000001EB5C1F98E0>, <__main__.Tuple object at 0x000001EB5C1F9640>, <__main__.Tuple object at 0x000001EB5C1F9C70>, <__main__.Tuple object at 0x000001EB5C1F94C0>, <__main__.Tuple object at 0x000001EB5C1F9F40>, <__main__.Tuple object at 0x000001EB5C1F9A90>, <__main__.Tu

### Grouping and Aggregation (Группировка и аггрегация)

**The Map Function:** Для каждого кортежа $(a, b, c$) создайте пару $(a, b)$.

**The Reduce Function:** Ключ представляет ту или иную группу. Примение аггрегирующую операцию $\theta$ к списку значений $[b1, b2, . . . , bn]$ ассоциированных с ключом $a$. Возвращайте в выходной поток $(a, x)$, где $x$ результат применения  $\theta$ к списку. Например, если $\theta$ это $SUM$, тогда $x = b1 + b2 + · · · + bn$, а если $\theta$ is $MAX$, тогда $x$ это максимальное из значений $b1, b2, . . . , bn$.

In [35]:
# Генерируем случайный кортеж из трех целых чисел от 0 до 3
def get_random_tuple():
    return (random.randint(0, 3), random.randint(0, 3), random.randint(0, 3))

# Генерируем список случайных кортежей заданного количества с помощью get_random_tuple
def RECORDREADER(count):
    return [get_random_tuple() for i in range(count)]

# Возвращаем кортеж из первых двух элементов входного кортежа
def MAP(t: Tuple):
    return (t[0], t[1])

# Возвращаем сумму всех значений в переданном списке
def tetta(values):
    return sum(values)

# Возвращаем ключ и результат функции tetta, примененной к значениям
def REDUCE(key, values: Iterator[NamedTuple]):
    x = tetta(values)  # Вычисляем сумму значений
    return (key, x)

def group_by_key(iterable):
    t = {}  # Создаем пустой словарь
    for (k2, v2) in iterable:  # Перебираем ключи и значения во входном итерируемом объекте
        t[k2] = t.get(k2, []) + [v2]  # Добавляем значение v2 в список, связанный с ключом k2 в словаре t
    return t.items()  # Возвращаем элементы словаря как список кортежей

record = RECORDREADER(100)  # Создаем список из 100 случайных записей
print(record)  # Выводим список записей

map_output = list(map(lambda x: MAP(x), RECORDREADER(100)))  # Применяем MAP к каждой записи и сохраняет результат
print(map_output)  # Выводим результаты отображения

shuffle_output = group_by_key(map_output)  # Группируем результаты отображения по ключу
shuffle_output = list(shuffle_output)  # Преобразуем результат группировки в список
print(shuffle_output)  # Выводим результат группировки

reduce_output = list(map(lambda x: REDUCE(*x), shuffle_output))  # Применяем REDUCE к результату группировки и сохраняет результат
print(reduce_output)  # Выводим редуцированный результат

[(3, 3, 0), (2, 1, 1), (3, 3, 3), (1, 2, 2), (1, 2, 0), (1, 0, 1), (2, 3, 3), (3, 1, 0), (2, 2, 2), (3, 0, 3), (2, 1, 0), (2, 3, 1), (2, 2, 2), (1, 0, 0), (2, 1, 0), (0, 0, 0), (2, 3, 1), (1, 2, 0), (0, 1, 1), (1, 3, 2), (3, 3, 0), (0, 2, 1), (0, 1, 0), (1, 1, 0), (3, 3, 0), (1, 0, 0), (3, 0, 1), (3, 1, 3), (1, 1, 0), (2, 3, 3), (2, 2, 0), (3, 3, 2), (3, 0, 1), (1, 2, 3), (2, 2, 0), (1, 2, 1), (3, 2, 1), (1, 2, 3), (0, 1, 2), (3, 1, 0), (2, 2, 2), (3, 1, 0), (0, 2, 3), (1, 3, 3), (3, 1, 0), (2, 3, 2), (3, 2, 1), (1, 0, 0), (0, 2, 1), (0, 3, 2), (3, 0, 0), (3, 2, 2), (1, 3, 0), (3, 3, 2), (3, 0, 1), (3, 2, 0), (1, 3, 2), (2, 3, 0), (2, 0, 1), (1, 2, 2), (0, 2, 2), (0, 2, 3), (0, 3, 3), (0, 0, 1), (2, 2, 2), (2, 3, 1), (2, 0, 0), (1, 3, 0), (2, 1, 2), (2, 1, 2), (3, 1, 3), (1, 2, 2), (1, 1, 1), (3, 3, 0), (3, 1, 2), (3, 3, 3), (0, 1, 3), (0, 2, 3), (1, 1, 1), (2, 2, 0), (3, 2, 2), (2, 3, 0), (0, 1, 3), (2, 1, 0), (2, 3, 3), (1, 1, 1), (1, 3, 0), (1, 0, 2), (1, 0, 3), (1, 2, 3), (1, 0, 3)

# 

### Matrix-Vector multiplication

Случай, когда вектор не помещается в памяти Map задачи


## Matrix multiplication (Перемножение матриц)

Если у нас есть матрица $M$ с элементами $m_{ij}$ в строке $i$ и столбце $j$, и матрица $N$ с элементами $n_{jk}$ в строке $j$ и столбце $k$, тогда их произведение $P = MN$ есть матрица $P$ с элементами $p_{ik}$ в строке $i$ и столбце $k$, где

$$p_{ik} =\sum_{j} m_{ij}n_{jk}$$

Необходимым требованием является одинаковое количество столбцов в $M$ и строк в $N$, чтобы операция суммирования по  $j$ была осмысленной. Мы можем размышлять о матрице, как об отношении с тремя атрибутами: номер строки, номер столбца, само значение. Таким образом матрица $M$ предстваляется как отношение $ M(I, J, V )$, с кортежами $(i, j, m_{ij})$, и, аналогично, матрица $N$ представляется как отношение $N(J, K, W)$, с кортежами $(j, k, n_{jk})$. Так как большие матрицы как правило разреженные (большинство значений равно 0), и так как мы можем нулевыми значениями пренебречь (не хранить), такое реляционное представление достаточно эффективно для больших матриц. Однако, возможно, что координаты $i$, $j$, и $k$ неявно закодированы в смещение позиции элемента относительно начала файла, вместо явного хранения. Тогда, функция Map (или Reader) должна быть разработана таким образом, чтобы реконструировать компоненты $I$, $J$, и $K$ кортежей из смещения.

Произведение $MN$ это фактически join, за которым следуют группировка по ключу и аггрегация. Таким образом join отношений $M(I, J, V )$ и $N(J, K, W)$, имеющих общим только атрибут $J$, создаст кортежи $(i, j, k, v, w)$ из каждого кортежа $(i, j, v) \in M$ и кортежа $(j, k, w) \in N$. Такой 5 компонентный кортеж представляет пару элементов матрицы $(m_{ij} , n_{jk})$. Что нам хотелось бы получить на самом деле, это произведение этих элементов, то есть, 4 компонентный кортеж$(i, j, k, v \times w)$, так как он представляет произведение $m_{ij}n_{jk}$. Мы представляем отношение как результат одной MapReduce операции, в которой мы можем произвести группировку и аггрегацию, с $I$ и $K$  атрибутами, по которым идёт группировка, и суммой  $V \times W$. 





In [36]:
# MapReduce model
def flatten(nested_iterable):
  for iterable in nested_iterable:
    for element in iterable:
      yield element

def groupbykey(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

def MapReduce(RECORDREADER, MAP, REDUCE):
  return flatten(map(lambda x: REDUCE(*x), groupbykey(flatten(map(lambda x: MAP(*x), RECORDREADER())))))

Реализуйте перемножение матриц с использованием модельного кода MapReduce для одной машины в случае, когда одна матрица хранится в памяти, а другая генерируется RECORDREADER-ом.

In [37]:
import numpy as np

I = 2
J = 3
K = 4*10

# Создаем двумерный массив small_mat размером IxJ, заполненный случайными значениями от 0 до 1
small_mat = np.random.rand(I, J)

# Создаем двумерный массив big_mat размером JxK, заполненный случайными значениями от 0 до 1
big_mat = np.random.rand(J, K)

def RECORDREADER():
    """
    Генератор, который перебирает индексы j и k в двумерном массиве big_mat
    и возвращает кортеж ((j, k), значение) для каждого элемента.
    """
    for j in range(big_mat.shape[0]):  # Перебираем индексы j
        for k in range(big_mat.shape[1]):  # Перебираем индексы k
            yield ((j, k), big_mat[j, k])  # Возвращаем кортеж ((j, k), значение элемента)

def MAP(k1, v1):
    """
    Функция отображения, которая для каждого элемента k1 с соответствующим значением v1
    вычисляет новые значения для каждого индекса i и k с использованием матрицы small_mat.
    """
    (j, k) = k1  # Разбиваем k1 на индексы j и k
    w = v1  # Присваиваем значение v1 переменной w
    for i in range(small_mat.shape[0]):  # Перебираем индексы i в small_mat
        yield ((i, k), w * small_mat[i][j])  # Возвращаем новые значения ((i, k), новое значение)

def REDUCE(key, values):
    """
    Функция редукции, которая суммирует значения из values для каждого ключа key и возвращает сумму.
    """
    (i, k) = key  # Разбиваем key на индексы i и k
    el_value = 0  # Инициализируем переменную для суммы значений
    for v in values:  # Перебираем значения в списке values
        el_value += v  # Суммируем значения
    yield ((i, k), el_value)  # Возвращаем кортеж ((i, k), сумма значений)

Проверьте своё решение

In [38]:
# CHECK THE SOLUTION
reference_solution = np.matmul(small_mat, big_mat) 
solution = MapReduce(RECORDREADER, MAP, REDUCE)

def asmatrix(reduce_output):
  reduce_output = list(reduce_output)
  I = max(i for ((i,k), vw) in reduce_output)+1
  K = max(k for ((i,k), vw) in reduce_output)+1
  mat = np.empty(shape=(I,K))
  for ((i,k), vw) in reduce_output:
    mat[i,k] = vw
  return mat

np.allclose(reference_solution, asmatrix(solution)) # should return true

True

In [39]:
reduce_output = list(MapReduce(RECORDREADER, MAP, REDUCE))
max(i for ((i,k), vw) in reduce_output)

1

Реализуйте перемножение матриц  с использованием модельного кода MapReduce для одной машины в случае, когда обе матрицы генерируются в RECORDREADER. Например, сначала одна, а потом другая.

In [40]:
import numpy as np

# Определяем размерности матриц
I = 2
J = 3
K = 4 * 10

# Генерируем случайные матрицы small_mat и big_mat
small_mat = np.random.rand(I, J)
big_mat = np.random.rand(J, K)

# Вычисляем эталонное решение путем перемножения матриц small_mat и big_mat
reference_solution = np.matmul(small_mat, big_mat)

def RECORDREADER():
    """
    Генератор, который сначала перебирает индексы i и j в small_mat и возвращает соответствующие элементы,
    а затем перебирает индексы j и k в big_mat и возвращает соответствующие элементы.
    """
    for i in range(small_mat.shape[0]):  # Перебираем индексы i в small_mat
        for j in range(small_mat.shape[1]):  # Перебираем индексы j в small_mat
            yield ((0, i, j), small_mat[i, j])  # Возвращаем кортеж ((0, i, j), значение элемента)

    for j in range(big_mat.shape[0]):  # Перебираем индексы j в big_mat
        for k in range(big_mat.shape[1]):  # Перебираем индексы k в big_mat
            yield ((1, j, k), big_mat[j, k])  # Возвращаем кортеж ((1, j, k), значение элемента)

def MAP_JOIN(k1, v1):
    """
    Функция отображения, которая разбивает входные данные на две группы: значения, соответствующие small_mat,
    и значения, соответствующие big_mat, и передает их дальше.
    """
    (mat_num, i, j) = k1  # Разбиваем ключ на матрицу, индекс i и индекс j
    w = v1  # Присваиваем значение v1 переменной w
    if mat_num == 0:  # Если матрица small_mat
        yield (j, (mat_num, i, w))  # Возвращаем кортеж (j, (mat_num, i, w))
    else:  # Иначе (матрица big_mat)
        yield (i, (mat_num, j, w))  # Возвращаем кортеж (i, (mat_num, j, w))

def REDUCE_JOIN(key, values):
    """
    Функция редукции, которая объединяет значения из двух групп (small_mat и big_mat)
    и возвращает их произведение для каждой комбинации индексов.
    """
    from_first_mat = [v for v in values if v[0] == 0]  # Фильтруем значения, соответствующие small_mat
    from_second_mat = [v for v in values if v[0] == 1]  # Фильтруем значения, соответствующие big_mat
    for f in from_first_mat:  # Перебираем значения из small_mat
        for s in from_second_mat:  # Перебираем значения из big_mat
            yield ((f[1], s[1]), f[2] * s[2])  # Возвращаем произведение значений

def MAP_MUL(k1, v1):
    """
    Функция отображения, которая просто возвращает ключ и значение без изменений.
    """
    (i, k) = k1  # Разбиваем ключ на индексы i и k
    yield ((i, k), v1)  # Возвращаем кортеж ((i, k), v1)

def REDUCE_MUL(key, values):
    """
    Функция редукции, которая суммирует все значения для каждого ключа и возвращает сумму.
    """
    res_el_value = 0  # Инициализируем переменную для суммы значений
    for v in values:  # Перебираем значения в списке values
        res_el_value += v  # Суммируем значения
    yield (key, res_el_value)  # Возвращаем ключ и сумму значений

def GET_JOINED():
    """
    Функция, которая возвращает объединенные результаты из joined.
    """
    for j in joined:  # Перебираем значения в joined
        yield j  # Возвращаем значение

# Вызываем MapReduce для выполнения операции JOIN
joined = MapReduce(RECORDREADER, MAP_JOIN, REDUCE_JOIN)
# Вызываем MapReduce для выполнения операции умножения
solution = MapReduce(GET_JOINED, MAP_MUL, REDUCE_MUL)
# Проверяем, что решение совпадает с эталонным решением с использованием np.allclose
np.allclose(reference_solution, asmatrix(solution))  # Должен вернуть True

True

Реализуйте перемножение матриц с использованием модельного кода MapReduce Distributed, когда каждая матрица генерируется в своём RECORDREADER. 

In [41]:
maps = 2
reducers = 2


def INPUTFORMAT():
    global maps

    # Функция генерирует пары элементов из обеих матриц
    def RECORDREADER(i_range):
        for i in i_range:  # Перебираем диапазон индексов i
            for j in range(J):  # Перебираем индексы j
                for k in range(K):  # Перебираем индексы k
                    yield (((i, j), small_mat[i, j]), ((j, k), big_mat[j, k]))  # Возвращаем пару элементов

    # Разделение между вычислительными узлами происходит по строкам первой матрицы
    split_size = int(np.ceil(I / maps))  # Вычисляем размер раздела
    for i in range(0, I, split_size):  # Перебираем диапазоны индексов i с шагом split_size
        yield RECORDREADER(range(i, i + split_size))  # Возвращаем результат работы функции RECORDREADER


# Функция возвращает произведения соответствующих элементов из обеих матриц
def MAP(element1, element2):
    (i, j), v1 = element1
    (j, k), v2 = element2

    yield ((i, k), v1 * v2)

# Функция суммирует полученные произведения
def REDUCE(key, values):
    (i, k) = key  # Разбиваем ключ на индексы i и k

    # solution code that yield(k3,v3) pairs
    k3 = (i, k)  # Создаем ключ k3
    v3 = 0  # Инициализируем переменную для суммы

    for j in range(J):  # Перебираем индексы j
        v3 += values[j]  # Суммируем значения

    yield (k3, v3)  # Возвращаем ключ и сумму значений


# try to set COMBINER=REDUCER and look at the number of values sent over the network
partitioned_output = MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, COMBINER=None)
partitioned_output = [
    (partition_id, list(partition)) for (partition_id, partition) in partitioned_output
]

# Соединяем полученные произведения с разных вычислительных узлов
solution = []
for output_part in partitioned_output:  # Перебираем выходные части
    for element in output_part[1]:  # Перебираем элементы выходной части
        solution += [element]  # Добавляем элемент в решение

print(asmatrix(solution))
np.allclose(reference_solution, asmatrix(solution))  # should return true

240 key-value pairs were sent over a network.
[[0.83567617 0.76693532 0.81421986 0.93286833 1.00666554 1.83237462
  1.35847856 1.24498094 0.6461546  1.1935875  1.2225548  1.6166573
  1.07840429 0.72278764 1.05590409 0.73956177 1.2292719  1.46588631
  1.01920025 1.63962525 0.99715144 0.92021516 1.33276329 1.01422579
  1.27844271 1.5351707  1.162331   1.37648443 1.14550149 1.32044081
  1.64658629 1.46963601 0.7948784  0.37241511 1.24062749 1.46342476
  1.33225307 0.91647748 1.1292259  1.30842477]
 [0.6481539  0.82275859 1.02460405 0.49050917 1.38046164 1.71176575
  1.41876763 1.27466542 0.76638174 1.01091276 1.56992386 1.41261776
  0.78381292 0.9317143  0.99068909 0.3961241  1.0424729  1.32863078
  0.9264703  1.40133257 0.99694374 1.13368199 1.02937632 1.09154591
  0.999332   1.20455139 1.20074963 1.2058232  0.96656847 1.1931205
  1.38166067 1.45013545 0.69263301 0.39110881 0.80162849 1.31770284
  0.97809765 1.07116544 1.32873559 1.03193674]]


True

Обобщите предыдущее решение на случай, когда каждая матрица генерируется несколькими RECORDREADER-ами, и проверьте его работоспособность. Будет ли работать решение, если RECORDREADER-ы будут генерировать случайное подмножество элементов матрицы?

In [42]:

# Определение размеров матриц
I = 2  # Количество строк первой матрицы и строк в итоговой матрице
J = 3  # Количество столбцов первой матрицы и строк второй матрицы
K = 4*10  # Количество столбцов второй матрицы и столбцов в итоговой матрице

# Генерация случайных матриц
small_mat = np.random.rand(I, J)  # Генерация первой матрицы
big_mat = np.random.rand(J, K)  # Генерация второй матрицы

# Получение эталонного решения через умножение матриц
reference_solution = np.matmul(small_mat, big_mat)

# Функция для "разглаживания" вложенных итерируемых объектов
def flatten(nested_iterable):
  for iterable in nested_iterable:
    for element in iterable:
      yield element

# Функция для группировки элементов по ключу
def groupbykey(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

# Функция для распределенной группировки элементов по ключу
def groupbykey_distributed(map_partitions, PARTITIONER):
  global reducers
  partitions = [dict() for _ in range(reducers)]
  for map_partition in map_partitions:
    for (k2, v2) in map_partition:
      p = partitions[PARTITIONER(k2)]
      p[k2] = p.get(k2, []) + [v2]
  return [(partition_id, sorted(partition.items(), key=lambda x: x[0])) for (partition_id, partition) in enumerate(partitions)]

# Функция для определения разделителя (по ключу)
def PARTITIONER(obj):
  global reducers
  return hash(obj) % reducers

# Функция для выполнения MapReduce на распределенных данных
def MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, PARTITIONER=PARTITIONER, COMBINER=None):
  map_partitions = map(lambda record_reader: flatten(map(lambda k1v1: MAP(*k1v1), record_reader)), INPUTFORMAT())

  if COMBINER != None:
    map_partitions = map(lambda map_partition: flatten(map(lambda k2v2: COMBINER(*k2v2), groupbykey(map_partition))), map_partitions)

  reduce_partitions = groupbykey_distributed(map_partitions, PARTITIONER)  # shuffle

  reduce_outputs = map(lambda reduce_partition: (reduce_partition[0], flatten(map(lambda reduce_input_group: REDUCE(*reduce_input_group), reduce_partition[1]))), reduce_partitions)

  print("{} key-value pairs were sent over a network.".format(sum([len(vs) for (k,vs) in flatten([partition for (partition_id, partition) in reduce_partitions])])))
  return reduce_outputs

# Функция для преобразования результата REDUCE в матрицу
def asmatrix(reduce_output):
  reduce_output = list(reduce_output)
  I = max(i for ((i,k), vw) in reduce_output) + 1
  K = max(k for ((i,k), vw) in reduce_output) + 1
  mat = np.empty(shape=(I,K))
  for ((i,k), vw) in reduce_output:
    mat[i,k] = vw
  return mat

# Генератор для вводных данных
def INPUTFORMAT():
  first_mat = []

  for i in range(small_mat.shape[0]):
    for j in range(small_mat.shape[1]):
      first_mat.append(((0, i, j), small_mat[i,j]))  # первая матрица

  global maps
  split_size = int(np.ceil(len(first_mat)/maps))

  for i in range(0, len(first_mat), split_size):
    yield first_mat[i:i+split_size]

  second_mat = []

  for j in range(big_mat.shape[0]):
    for k in range(big_mat.shape[1]):
      second_mat.append(((1, j, k), big_mat[j,k]))  # вторая матрица

  split_size = int(np.ceil(len(second_mat)/maps))

  for i in range(0, len(second_mat), split_size):
    yield second_mat[i:i+split_size]

# MAP функция для соединения матриц
def MAP_JOIN(k1, v1):
  (mat_num, i, j) = k1
  w = v1

  if mat_num == 0:
    yield (j, (mat_num, i, w))

  else:
    yield (i, (mat_num, j, w))

# REDUCE функция для соединения матриц
def REDUCE_JOIN(key, values):
  from_first_mat = [v for v in values if v[0] == 0]
  from_second_mat = [v for v in values if v[0] == 1]

  for f in from_first_mat:
    for s in from_second_mat:
      yield ((f[1], s[1]), f[2] * s[2])

# Генератор для получения соединенных данных
def GET_JOINED():

  for j in joined:
    print("aa", j)
    yield j[1]

# MAP функция для умножения значений
def MAP_MUL(k1, v1):
  yield (k1, v1)

# REDUCE функция для умножения значений
def REDUCE_MUL(key, values):
  res_val = 0

  for v in values:
    res_val += v
  yield (key, res_val)

maps = 3
reducers = 2

# Выполнение MapReduce для соединения матриц
partitioned_output = MapReduceDistributed(INPUTFORMAT, MAP_JOIN, REDUCE_JOIN, COMBINER=None)
joined = [(partition_id, list(partition)) for (partition_id, partition) in partitioned_output]
print(joined)

# Выполнение MapReduce для умножения значений
mul_output = MapReduceDistributed(GET_JOINED, MAP_MUL, REDUCE_MUL, COMBINER=None)
pre_result = [(partition_id, list(partition)) for (partition_id, partition) in mul_output]
print(pre_result)

solution = []

for p in pre_result:

  for v in p[1]:
    solution.append(v)

print(solution)

np.allclose(reference_solution, asmatrix(solution))

126 key-value pairs were sent over a network.
[(0, [((0, 0), 0.1033323754474534), ((0, 1), 0.0038716376954038988), ((0, 2), 0.05787833364527408), ((0, 3), 0.11151757906189498), ((0, 4), 0.046381776247769006), ((0, 5), 0.035108402145897916), ((0, 6), 0.009595321332611388), ((0, 7), 0.09379160997984201), ((0, 8), 0.06943884204794325), ((0, 9), 0.008148119371027622), ((0, 10), 0.03647470159042513), ((0, 11), 0.05313259203388244), ((0, 12), 0.0916506195772022), ((0, 13), 0.028572245928828826), ((0, 14), 0.10294011745876366), ((0, 15), 0.1176590246470005), ((0, 16), 0.06360645237677391), ((0, 17), 0.06864202088405523), ((0, 18), 0.07495383927825283), ((0, 19), 0.041828560070764895), ((0, 20), 0.06724910567573196), ((0, 21), 0.01607322740961028), ((0, 22), 0.10881835933859561), ((0, 23), 0.08331390747285689), ((0, 24), 0.09829787034368594), ((0, 25), 0.016289422841245633), ((0, 26), 0.07601933155159998), ((0, 27), 0.08461335087042145), ((0, 28), 0.00296924636877737), ((0, 29), 0.011216311431

True

Решение будет работать, так как каждая пара элементов, генерируемая RECORDREADER, идентифицируется по ее индексам